TODO: 
replace weird characters quotation marks
nltk stop words remove them

not sure if need it, but normalization: lowercase, stemming, lemmatization

# Infrastructure

In [91]:
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [107]:
txt_folder = './data/original/txts/'
txt_adjusted_folder = './data/original/txts/'

In [1]:
my_file = open(txt_folder + 'the_dunwich_horror.txt')
raw_text = my_file.read()

# Preparation

let's check what kind of characters we have

In [57]:
characters = raw_text.lower().replace('\n','')
characters = characters.replace('\x0c','')
characters = characters.replace(' ','')

In [70]:
unique_characters = set(characters)
unique_characters_in_string = ''.join(set(characters))
unique_characters_in_string

'-2wdj0i”m8uüosczävèr‖l)q,4b76e’n—a1(:é.ftpk?x‘hg!;93―y5'

In [81]:
regular_characters = 'abcdefghijklmnopqrstuvwxyz1234567890.,?!()-:;\'"'

In [82]:
for character in regular_characters:
    try:
        unique_characters.remove(character)
    except:
        pass

In [83]:
unique_characters

{'ä', 'è', 'é', 'ü', '—', '―', '‖', '‘', '’', '”'}

In [80]:
'—' == '―'

False

In [84]:
'‘' == '’'

False

In [85]:
'‘' == "'" or  '’' == "'"

False

In [59]:
raw_text.index('ä')

36016

In [86]:
raw_text.index('è')

32285

In [88]:
raw_text.index('é')

66610

In [90]:
raw_text.index('ü')

66781

# Word Count

Simply count the words, can use string split for that. 
This will for example combine end-of-sentence punctuations, but that's fine. 

In [8]:
words = raw_text.split()

In [10]:
len(words)

17435

this would not work, leaves in dashes, like 'superstition—but'

In [22]:
words = re.split(r'[-\s.,;—!‖?]+', raw_text)

In [23]:
len(words)

17607

# Tokenize with NLTK

different tokenizations, do page 41 example do do things like word that in every writing

trying regex tokenizer first

In [29]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+|$[0-9.]+|\S+')
words = tokenizer.tokenize(raw_text)

In [30]:
len(words)

19945

source: http://www.nltk.org/book/ch01.html
https://www.nltk.org/book/ch03.html

In [32]:
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
tokens = tokenizer.tokenize(raw_text)

In [33]:
len(tokens)

19408

In [34]:
tokens[500:]

['close',
 'to',
 'the',
 'feet',
 'of',
 'the',
 'domed',
 'hills',
 'among',
 'which',
 'it',
 'rises.',
 'As',
 'the',
 'hills',
 'draw',
 'nearer',
 ',',
 'one',
 'heeds',
 'their',
 'wooded',
 'sides',
 'more',
 'than',
 'their',
 'stone-crowned',
 'tops.',
 'Those',
 'sides',
 'loom',
 'up',
 'so',
 'darkly',
 'and',
 'precipitously',
 'that',
 'one',
 'wishes',
 'they',
 'would',
 'keep',
 'their',
 'distance',
 ',',
 'but',
 'there',
 'is',
 'no',
 'road',
 'by',
 'which',
 'to',
 'escape',
 'them.',
 'Across',
 'a',
 'covered',
 'bridge',
 'one',
 'sees',
 'a',
 'small',
 'village',
 'huddled',
 'between',
 'the',
 'stream',
 'and',
 'the',
 'vertical',
 'slope',
 'of',
 'Round',
 'Mountain',
 ',',
 'and',
 'wonders',
 'at',
 'the',
 'cluster',
 'of',
 'rotting',
 'gambrel',
 'roofs',
 'bespeaking',
 'an',
 'earlier',
 'architectural',
 'period',
 'than',
 'that',
 'of',
 'the',
 'neighbouring',
 'region.',
 'It',
 'is',
 'not',
 'reassuring',
 'to',
 'see',
 ',',
 'on',
 'a',

let's see what it does with a sentence like 'I dun‘t keer what folks think—ef Lavinny‘s boy looked like his pa, he wouldn‘t look like nothin‘ ye expeck.'

In [39]:
tokens.index('Lavinny')

2169

In [40]:
tokens[2169:]

['Lavinny',
 '‘',
 's',
 'boy',
 'looked',
 'like',
 'his',
 'pa',
 ',',
 'he',
 'wouldn',
 '‘',
 't',
 'look',
 'like',
 'nothin',
 '‘',
 'ye',
 'expeck.',
 'Ye',
 'needn',
 '‘',
 't',
 'think',
 'the',
 'only',
 'folks',
 'is',
 'the',
 'folks',
 'hereabaouts.',
 'Lavinny',
 '‘',
 's',
 'read',
 'some',
 ',',
 'an',
 '‘',
 'has',
 'seed',
 'some',
 'things',
 'the',
 'most',
 'o',
 '‘',
 'ye',
 'only',
 'tell',
 'abaout.',
 'I',
 'calc',
 '‘',
 'late',
 'her',
 'man',
 'is',
 'as',
 'good',
 'a',
 'husban',
 '‘',
 'as',
 'ye',
 'kin',
 'find',
 'this',
 'side',
 'of',
 'Aylesbury',
 ';',
 'an',
 '‘',
 'ef',
 'ye',
 'knowed',
 'as',
 'much',
 'abaout',
 'the',
 'hills',
 'as',
 'I',
 'dew',
 ',',
 'ye',
 'wouldn',
 '‘',
 't',
 'ast',
 'no',
 'better',
 'church',
 'weddin',
 '‘',
 'nor',
 'her',
 '‘',
 'n.',
 'Let',
 'me',
 'tell',
 'ye',
 'suthin',
 '‘',
 '—some',
 'day',
 'yew',
 'folks',
 '’',
 'll',
 'hear',
 'a',
 'child',
 'o',
 '’',
 'Lavinny',
 '’',
 's',
 'a-callin',
 '’',
 'i

In [41]:
test_sentence = "I don't think we'll go."

In [42]:
tokenizer.tokenize(test_sentence)

['I', 'do', "n't", 'think', 'we', "'ll", 'go', '.']

In [47]:
test_sentence_2 = "Why wouldn't we?"

In [48]:
tokenizer.tokenize(test_sentence_2)

['Why', 'would', "n't", 'we', '?']

In [50]:
test_sentence_3 = "Why wouldn‘t we?"

In [51]:
tokenizer.tokenize(test_sentence_3)

['Why', 'wouldn', '‘', 't', 'we', '?']

In [5]:
from nltk.util import ngrams

In [17]:
two_grams = list(ngrams(tokens,2))

In [21]:
my_dict = {x:two_grams.count(x) for x in two_grams}

In [22]:
sorted_d = sorted(my_dict.items(), key=lambda x: x[1])

In [24]:
sorted_d[-20:]

[(('o', '‘'), 26),
 (('of', 'a'), 29),
 (('It', 'was'), 30),
 ((',', 'the'), 31),
 (('seemed', 'to'), 31),
 ((';', 'and'), 33),
 (('at', 'the'), 38),
 ((',', 'an'), 42),
 (('from', 'the'), 46),
 (('‘', 't'), 46),
 ((',', 'but'), 54),
 (('on', 'the'), 58),
 (('to', 'the'), 59),
 (('and', 'the'), 64),
 (('.', '.'), 78),
 (('an', '‘'), 80),
 (('in', 'the'), 85),
 (('‘', 's'), 135),
 (('of', 'the'), 170),
 ((',', 'and'), 233)]

trying out things from nltk guide, need to convert to nltk text, based on forum here: https://stackoverflow.com/questions/29110950/python-concordance-command-in-nltk

maybe do Porter Stemmer (pg 59) for word analysis

# Other NLTK Stuff

ply with concordance and similar

In [28]:
textList = Text(tokens)
textList.concordance('horror')

Displaying 24 of 24 matches:
The Dunwich Horror ―Gorgons , and Hydras , and Chimaer
ble , and since a certain season of horror all the signboards pointing toward 
 our sensible age—since the Dunwich horror of 1928 was hushed up by those who 
ave the facts concerning the recent horror , can say just what is the matter w
ded only in 1928 , when the Dunwich horror came and went ; yet at no time did 
 was thought very notable until the horror of 1928 suggested the most valid of
some terrible part of the intruding horror , and to glimpse a hellish advance 
tains that May-Night ? What Roodmas horror fastened itself on the world in hal
as Wilbur Whateley. VI. The Dunwich horror itself came between Lammas and the 
 the dark of September 9th that the horror broke loose. The hill noises had be
and Cold Spring Glen , examining in horror the vast , monstrous prints , the m
end and beard the unknown Cyclopean horror in its lair. Three dogs that were w
re were many who hoped that the new horror had 

In [32]:
textList.similar('unseen')

dunwich new recent serene intruding mountainous


# Sentiment Analysis

https://github.com/cjhutto/vaderSentiment

In [92]:
sentiment_analyzer = SentimentIntensityAnalyzer()

In [95]:
type(sentiment_analyzer.lexicon)

dict

In [96]:
text_to_analyze = 'As the White Ship sailed silently away from the templed terraces of Zar, we beheld on the distant horizon ahead the spires of a mighty city; and the bearded man said to me: ―This is Thalarion, the City of a Thousand Wonders, wherein reside all those mysteries that man has striven in vain to fathom.‖ And I looked again, at closer range, and saw that the city was greater than any city I had known or dreamed of before. Into the sky the spires of its temples reached, so that no man might behold their peaks; and far back beyond the horizon stretched the grim, grey walls, over which one might spy only a few roofs, weird and ominous, yet adorned with rich friezes and alluring sculptures. I yearned mightily to enter this fascinating yet repellent city, and besought the bearded man to land me at the stone pier by the huge carven gate Akariel; but he gently denied my wish, saying: ―Into Thalarion, the City of a Thousand Wonders, many have passed but none returned. Therein walk only daemons and mad things that are no longer men, and the streets are white with the unburied bones of those who have looked upon the eidolon Lathi, that reigns over the city.‖ So the White Ship sailed on past the walls of Thalarion, and followed for many days a southward-flying bird, whose glossy plumage matched the sky out of which it had appeared.'



In [101]:
text_to_analyze_2 = 'Then came we to a pleasant coast gay with blossoms of every hue, where as far inland as we could see basked lovely groves and radiant arbours beneath a meridian sun. '



In [103]:
text_to_analyze_3 = 'At length they emerged on a muddy road to find the sun coming out. They were a little beyond the Seth Bishop place, but bent trees and hideously unmistakable tracks shewed what had passed by. Only a few moments were consumed in surveying the ruins just around the bend. It was the Frye incident all over again, and nothing dead or living was found in either of the collapsed shells which had been the Bishop house and barn. No one cared to remain there amidst the stench and tarry stickiness, but all turned instinctively to the line of horrible prints leading on toward the wrecked Whateley farmhouse and the altar-crowned slopes of Sentinel Hill.'


In [102]:
vs = sentiment_analyzer.polarity_scores(text_to_analyze_2)
vs

{'neg': 0.0, 'neu': 0.733, 'pos': 0.267, 'compound': 0.8807}

In [104]:
vs = sentiment_analyzer.polarity_scores(text_to_analyze_3)
vs

{'neg': 0.161, 'neu': 0.804, 'pos': 0.035, 'compound': -0.9504}

funny thing is: we can do this for the whole text, and list them best to worst

In [105]:
vs = sentiment_analyzer.polarity_scores(raw_text)
vs

{'neg': 0.106, 'neu': 0.826, 'pos': 0.068, 'compound': -1.0}

In [108]:
my_file = open(txt_folder + 'the_white_ship.txt')
raw_text_2 = my_file.read()

In [109]:
vs = sentiment_analyzer.polarity_scores(raw_text_2)
vs

{'neg': 0.059, 'neu': 0.82, 'pos': 0.122, 'compound': 0.9996}